In [53]:
import pandas as pd
import cv2
import os
import random
import math
from tqdm import tqdm

In [54]:
video_path = "/home/theo/Documents/Unif/Master/ChimpRec/ChimpRec-Dataset/CCR/videos"
annotations_path = "/home/theo/Documents/Unif/Master/ChimpRec/ChimpRec-Dataset/CCR/metadata/annotations/body_data.csv"
dataset_path = "/home/theo/Documents/Unif/Master/ChimpRec/ChimpRec-Dataset/Chimpanzee_detection_dataset"

In [55]:
df = pd.read_csv(annotations_path)
print(df.loc[df["video"] == "17.mp4"])

         year   video  frame  track         x         y         w         h  \
1652518  2013  17.mp4    957      3  0.143538  0.159027  0.020601  0.072732   
1652519  2013  17.mp4    958      3  0.143447  0.159946  0.020592  0.072688   
1652520  2013  17.mp4    959      3  0.143356  0.160865  0.020582  0.072644   
1652521  2013  17.mp4    960      3  0.143264  0.161785  0.020573  0.072600   
1652522  2013  17.mp4    961      3  0.143173  0.162704  0.020564  0.072557   
...       ...     ...    ...    ...       ...       ...       ...       ...   
1661293  2013  17.mp4   6526     16  0.098948  0.572017  0.034380  0.059375   
1661294  2013  17.mp4   6527     16  0.098759  0.570864  0.034378  0.061506   
1661295  2013  17.mp4   6528     16  0.097701  0.570297  0.035872  0.060066   
1661296  2013  17.mp4   6529     16  0.099128  0.570533  0.034480  0.060810   
1661297  2013  17.mp4   6530     16  0.098881  0.570130  0.034788  0.060924   

            label  
1652518  NEGATIVE  
1652519  NE

In [56]:
proportion_kept = 0.0015 # approximately 1500 images

In [57]:
# @input: n: size of the sample
# proportion: (proportion*100)% of the numbers in [0, n-1]
# @outputs
# No numbers in common between the outputs
def segment_dataset(n, proportion):
    # Calculate the number of elements to select based on the proportion
    num_elements = int(n * proportion)

    # Generate evenly spaced numbers within the range [0, n-1]
    step = n / num_elements
    selected_numbers = [int(i * step) for i in range(num_elements)]
    
    return selected_numbers

In [58]:
def get_len_video(path_to_video):
    video = cv2.VideoCapture(f"{video_path}/{path_to_video}")

    if not video.isOpened():
        print(f"Could not open video: {path_to_video}")
        return 0
    
    length = int(video.get(cv2.CAP_PROP_FRAME_COUNT))
    video.release()
    return length

In [59]:
def preprocess_coordinates(coord, img_width, img_height):

    x, y, w, h = coord
    
    x1 = int(x*img_width)
    y1 = int(y*img_width)
    x2 = int(x*img_width+w*img_width)
    y2 = int((y*img_width)+h*img_height)

    cx = (x1 + x2) / 2
    cy = (y1 + y2) / 2

    width = x2 - x1
    height = y2 - y1

    cx_norm = cx / img_width
    cy_norm = cy / img_height

    width_norm = width / img_width
    height_norm = height / img_height

    return [cx_norm, cy_norm, width_norm, height_norm]

In [60]:
def extract_metadata(video_type, video_id, frame_number, img_dim):
    img_W, img_H = img_dim
    sample_id = video_id.replace(".mp4", "")
    sub_df = df.loc[(df["video"] == video_id) & (df["frame"] == frame_number)]
    output_file = f"{dataset_path}/labels/{video_type}/{sample_id}_{frame_number}.txt"

    string_output = ""

    for index, row in sub_df.iterrows():
        x, y, w, h = row['x'], row['y'], row['w'], row['h']
        cx, cy, W, H = preprocess_coordinates((x, y, w, h), img_W, img_H)
        string_output += f"\n0 {cx} {cy} {W} {H}"

    with open(output_file, "w") as file:
        file.write(string_output.strip())
    file.close()
    
    


In [61]:
# @input:
# type: "train", "val" or "test" --> indicates which part of the dataset it is
# sample_id: name of the video with the extension
# idxs: indexes of the images to keep
# @output:
# nothing, the image are saved in the output folder
def extract_data(type, video_id):

    sample_id = video_id[:-4]

    output_folder = f"{dataset_path}/images/{type}"
    video = cv2.VideoCapture(f"{video_path}/{video_id}")
    len_video = get_len_video(video_id)

    idxs = segment_dataset(len_video, proportion_kept)
    idxs = sorted(idxs)

    for frame_count in idxs:
        df_ = df.loc[df["video"] == video_id]
        df_ = df_.loc[df_["frame"] == frame_count] # .loc[df["label"] != "NOTCHIMP"]

        # Set the video to the specific frame
        video.set(cv2.CAP_PROP_POS_FRAMES, frame_count)
        success, frame = video.read()

        if success and df_.size != 0:
            frame_filename = f"{output_folder}/{sample_id}_{frame_count}.jpg"
            cv2.imwrite(frame_filename, frame)
            
            # extract the annotations
            extract_metadata(type, video_id, frame_count, (frame.shape[1], frame.shape[0]))
        # else:
        #     print(f"Could not read frame {frame_count} in video {video_id}")
        #     print(df_)

    # Release the video capture object
    video.release()

In [ ]:
filenames = os.listdir(video_path)
n_files = len(filenames)

train_prop = 0.85

numbers = list(range(n_files))
random.shuffle(numbers)

train_video_idxs = numbers[:math.ceil(train_prop*n_files)]
val_video_idxs = numbers[math.ceil(train_prop*n_files):]

def process_dataset(video_idxs, type):
    # progression bar added
    for video_idx in tqdm(video_idxs, desc=f"Processing videos: {type}", colour="green"):
        video_id = filenames[video_idx]
        # try:
        #     extract_data(type, video_id)
        # except:
        #     print(f"Error: {video_id}")
        extract_data(type, video_id)
        

        

In [63]:
process_dataset(train_video_idxs, "train")
process_dataset(val_video_idxs, "val")

Processing videos: train:   0%|          | 0/23 [00:00<?, ?it/s]

20120201091719.mp4


Processing videos: train:   4%|▍         | 1/23 [00:04<01:46,  4.82s/it]

20120121160715.mp4


Processing videos: train:   9%|▊         | 2/23 [00:16<03:10,  9.09s/it]

...

Processing videos: val:  50%|█████     | 2/4 [00:06<00:07,  3.55s/it]

13.mp4


Processing videos: val:  75%|███████▌  | 3/4 [00:10<00:03,  3.84s/it]

14.mp4


Processing videos: val: 100%|██████████| 4/4 [00:14<00:00,  3.59s/it]
